In [2]:
import dask.dataframe as dd
import dask
import numpy as np
from dask import delayed
import dask_image.imread
import dask_image.ndfilters
import dask_image.ndmeasure
import cv2
from glob import glob
import json
import dask.array as da
import time
import xgboost as xgb
import pandas as pd
import os

@delayed
def padding(data, array_len, col_len):
    pad = np.zeros((array_len, col_len))
    length = min(array_len, len(data))
    pad[:length] = data[:length]
    return pad

@delayed
def img_resize(img):
    img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
    img = img.astype(np.float32)/255
    return img

path = os.getcwd()
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
# print(cluster,client)
# client.close()

C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52853 instead
  warnings.warn(


In [3]:
#making_label_set
# 
crop = {'1':'딸기','2':'토마토','3':'파프리카','4':'오이','5':'고추','6':'시설포도'}
disease = {'1':{'a1':'딸기잿빛곰팡이병','a2':'딸기흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '2':{'a5':'토마토흰가루병','a6':'토마토잿빛곰팡이병','b2':'열과','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '3':{'a9':'파프리카흰가루병','a10':'파프리카잘록병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '4':{'a3':'오이노균병','a4':'오이흰가루병','b1':'냉해피해','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '5':{'a7':'고추탄저병','a8':'고추흰가루병','b3':'칼슘결핍','b6':'다량원소결핍 (N)','b7':'다량원소결핍 (P)','b8':'다량원소결핍 (K)'},
           '6':{'a11':'시설포도탄저병','a12':'시설포도노균병','b4':'일소피해','b5':'축과병'}}
risk = {'1':'초기','2':'중기','3':'말기'}


#ensemble_labler
label_description = {}
for key, value in disease.items():
    label_description[f'{key}_00_0'] = f'{crop[key]}_정상'
    for disease_code in value:
        for risk_code in risk:
            label = f'{key}_{disease_code}_{risk_code}'
            label_description[label] = f'{crop[key]}_{disease[key][disease_code]}_{risk[risk_code]}'

global ensemble_label_encoder
ensemble_label_encoder = {key:idx for idx, key in enumerate(label_description)}
ensemble_label_decoder = {val:key for key, val in ensemble_label_encoder.items()}

#crop_labler
crop_label_description = {}
for key, value in disease.items():
    crop_label_description[f'{key}'] = f'{crop[key]}'
    
global crop_label_encoder
crop_label_encoder = {key:idx for idx, key in enumerate(crop_label_description)}
crop_label_decoder = {val:key for key, val in crop_label_encoder.items()}


#disease_labler
disease_label_description = {}
for key, value in disease.items():
    disease_label_description[f'00'] = f'{crop[key]}'
    for disease_code in value:
        label = f'{disease_code}'
        disease_label_description[label] = f'{disease_code}'

global disease_label_encoder
disease_label_encoder = {key:idx for idx, key in enumerate(disease_label_description)}
disease_label_decoder = {val:key for key, val in disease_label_encoder.items()}


#risk_labler
risk_label_description = {}
for key, value in risk.items():
    label = f'{key}'
    risk_label_description[label] = f'{key}'

global risk_label_encoder
risk_label_encoder = {key:idx for idx, key in enumerate(risk_label_description)}
risk_label_decoder = {val:key for key, val in risk_label_encoder.items()}





In [4]:
@delayed
def label_encoding(label):
    global label_encoder
    encoded_label = ensemble_label_encoder[label]
    return encoded_label

def getlable(jsonpath):
    with open(jsonpath, 'r') as f:
        json_file = json.load(f)

    crop = json_file['annotations']['crop']
    disease = json_file['annotations']['disease']
    risk = json_file['annotations']['risk']
    label = f'{crop}_{disease}_{risk}'
    return label
    
def get_label_list(labelpath_list):
    labelarr = np.array([])
    # labelarr = da.array([])
    for ind,json_path in enumerate(labelpath_list):
        # label = label_encoding(getlable(json_path))
        # label = da.array(np.array(label_encoding(getlable(json_path))))
        label = np.array(label_encoding(getlable(json_path)))
        labelarr = np.append(labelarr,label)
    return labelarr

데이터준비

In [5]:
#making_img_set
row_img = dask_image.imread.imread(path + "\\data_1\\train\\*\\*jpg")
datas = [img_resize(part) for part in row_img]
processed_img = np.array(dask.compute(*datas))
print(processed_img.shape)

(5767, 224, 224, 3)


In [6]:
label_list = sorted(glob(path + "\\data_1\\train\\*\\*.json"))
y_train = get_label_list(label_list)
# y_train = y_train.rechunk(5767)
results = dask.compute(*y_train)
label = np.array(results)
# print(label.shape)

In [7]:
label.shape, processed_img.shape

((5767,), (5767, 224, 224, 3))

모델 학습

In [8]:
processed_img[:,:,0,0]
# f'{file}\\{file_name}.csv'

(5767, 224)

In [ ]:
processed_img[:,:,0,1]

In [ ]:
processed_img[:,:,0,2]

In [21]:
np.concatenate((processed_img[:,:,0,0],processed_img[:,:,0,1],processed_img[:,:,0,2]),axis = 1)

array([[0.2627451 , 0.2627451 , 0.26666668, ..., 0.7490196 , 0.7529412 ,
        0.7607843 ],
       [0.73333335, 0.72156864, 0.7294118 , ..., 0.827451  , 0.80784315,
        0.79607844],
       [0.40784314, 0.4392157 , 0.5058824 , ..., 0.54509807, 0.58431375,
        0.8745098 ],
       ...,
       [0.64705884, 0.7294118 , 0.79607844, ..., 0.19215687, 0.20784314,
        0.21960784],
       [0.73333335, 0.7372549 , 0.75686276, ..., 0.34117648, 0.2784314 ,
        0.17254902],
       [0.9137255 , 0.89411765, 0.8235294 , ..., 0.5137255 , 0.5176471 ,
        0.5254902 ]], dtype=float32)

In [22]:
params = {"tree_method": "gpu_hist", 
          "objective": 'multi:softmax',
          "subsample": 0.8,
          "colsample_bytree": 0.8,
        #   "single_precision_histogram" : True,
          }
modelnames = []
for i in range(len(processed_img[0,0,:,0])):
          print("start")
          start = time.time()
          temptrain = np.concatenate((processed_img[:,:,i,0],processed_img[:,:,i,1],processed_img[:,:,i,2]),axis = 1)
          globals()[f"XGB_{i}"] = xgb.XGBClassifier(params,eval_metric='mlogloss',).fit(temptrain,label)
          print(time.time()-start)
          modelnames.append(f"XGB_{i}")

start


C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\core.py:499: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
C:\Users\rhqud\AppData\Local\Programs\Python\Python38\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


39.15878510475159
start
51.95165944099426
start
55.52392029762268
start
57.34278607368469
start
59.00073504447937
start
60.7701370716095
start
60.69258666038513
start
59.24736452102661
start
60.95941758155823
start
61.50640296936035
start


KeyboardInterrupt: 

In [ ]:
-